In [1]:
# %pip install requests

In [2]:
import requests
import json
import time
import os
import math

In [3]:
secrets_file = "secrets.json"

if not os.path.exists(secrets_file):
    raise FileNotFoundError(f"{secrets_file} not found. Please create the file with your API key.")

with open(secrets_file, "r") as file:
    secrets = json.load(file)

API_KEY = secrets.get("google_api_key")

if not API_KEY:
    raise ValueError("API key not found in secrets.json. Please add your key under 'google_api_key'.")

In [4]:
BASE_URL = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


In [14]:
# Bounding box for Calgary (approximate)
# Southwest corner: 50.8429, -114.3217
# Northeast corner: 51.1895, -113.8573
SW_LAT, SW_LNG = 50.8429, -114.3217
NE_LAT, NE_LNG = 51.1895, -113.8573

# length: 38.52km=~39km, width: 36.20km
LENGTH = 39 #km
WIDTH = 37 # km
GRID_SIZE = 0.5 # km

GRID_DIV_LENGTH = int(LENGTH  / GRID_SIZE)
GRID_DIV_WIDTH =  int(WIDTH  / GRID_SIZE)
# Separate grid into 500m x 500m grids
# Grid division parameters
GRID_DIVISIONS = 78 # Number of divisions in each direction
RADIUS = math.ceil(math.sqrt(2*(float(GRID_SIZE) * 1000 / 2)**2))   # Search radius in meters for each grid cell

# Generate grid of center points
lat_step = (NE_LAT - SW_LAT) / GRID_DIV_WIDTH
lng_step = (NE_LNG - SW_LNG) / GRID_DIV_LENGTH
grid_centers = [
    (SW_LAT + i * lat_step, SW_LNG + j * lng_step)
    for i in range(GRID_DIV_WIDTH)
    for j in range(GRID_DIV_LENGTH)
]

print(RADIUS)
print(len(grid_centers))

354
5772


In [6]:
def fetch_results(url, params):
    """
    Fetches data from the given URL with provided parameters.
    """
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response}")
        return None

In [7]:
# Collect all place IDs
all_place_ids = set()
all_places = []
queries = 0
grid = 0
for center in grid_centers:

    print(f"Querying center: {center}")
    params = {
        "location": f"{center[0]},{center[1]}",
        "radius": RADIUS,
        "type": "restaurant",
        "key": API_KEY
    }
    grid += 1
    next_page_token = None
    while True:
        if next_page_token:
            params["pagetoken"] = next_page_token
            time.sleep(2)  # Delay as per Google API guidelines

        data = fetch_results(BASE_URL, params)
        queries += 1
        print("Grid Number: ", grid)
        print("Query Number: ", queries)

        if data['status'] == 'OK':
            all_places.extend(data['results'])

            for place in data["results"]:
                all_place_ids.add(place["place_id"])

            output_file = "resturaunt_ids.json"
            with open(output_file, "w", encoding="utf-8") as f:
                json.dump(list(all_place_ids), f, indent=4)

            output_file = "resturaunts_details.json"
            with open(output_file, "w", encoding="utf-8") as f:
                json.dump(list(all_places), f, indent=4)

            if "next_page_token" in data:
                next_page_token = data.get("next_page_token")
            else:
                break
        else:
            break

Querying center: (50.8429, -114.3217)
Grid Number:  1
Query Number:  1
Querying center: (50.8429, -114.31574615384616)
Grid Number:  2
Query Number:  2
Querying center: (50.8429, -114.30979230769232)
Grid Number:  3
Query Number:  3
Querying center: (50.8429, -114.30383846153846)
Grid Number:  4
Query Number:  4
Querying center: (50.8429, -114.29788461538462)
Grid Number:  5
Query Number:  5
Querying center: (50.8429, -114.29193076923077)
Grid Number:  6
Query Number:  6
Querying center: (50.8429, -114.28597692307693)
Grid Number:  7
Query Number:  7
Querying center: (50.8429, -114.28002307692309)
Grid Number:  8
Query Number:  8
Querying center: (50.8429, -114.27406923076924)
Grid Number:  9
Query Number:  9
Querying center: (50.8429, -114.26811538461538)
Grid Number:  10
Query Number:  10
Querying center: (50.8429, -114.26216153846154)
Grid Number:  11
Query Number:  11
Querying center: (50.8429, -114.2562076923077)
Grid Number:  12
Query Number:  12
Querying center: (50.8429, -114.2

In [8]:
output_file = "resturaunt_ids.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(list(all_place_ids), f, indent=4)

output_file = "resturaunts_details.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(list(all_places), f, indent=4)

In [16]:
def calculate_grid_centers(sw_lat, sw_lng, ne_lat, ne_lng, grid_divisions):
    # Generate grid of center points
    lat_step = (ne_lat - sw_lat) / grid_divisions
    lng_step = (ne_lng - sw_lng) / grid_divisions
    grid_centers = [
        (sw_lat + i * lat_step, sw_lat + j * lng_step)
        for i in range(grid_divisions)
        for j in range(grid_divisions)
    ]
    return grid_centers

In [22]:
# grids with potentially more than 60 resturaunts:
big_grids = [(50.99746486486487, -114.07163846153846),
(51.034935135135136, -114.07163846153846),
(51.03961891891892, -114.09545384615384),
(51.03961891891892, -114.0895),
(51.03961891891892, -114.08354615384616),
(51.03961891891892, -114.07759230769231),
(51.03961891891892, -114.07163846153846),
(51.03961891891892, -114.06568461538461),
(51.0443027027027, -114.07759230769231),
(51.0443027027027, -114.07163846153846),
(51.0443027027027, -114.06568461538461),
(51.0443027027027, -114.05973076923077),
(51.04898648648649, -114.08354615384616),
(51.04898648648649, -114.07759230769231),
(51.04898648648649, -114.07163846153846),
(51.04898648648649, -114.06568461538461),
(51.04898648648649, -114.05973076923077),
(51.053670270270274, -114.0895),
(51.053670270270274, -114.08354615384616),
(51.10050810810811, -113.96446923076923),]

micro_centers = []
for i, j in big_grids:
    ne_lat = i + (lat_step /2)
    ne_lng = j + (lng_step /2)
    sw_lat = i - (lat_step /2)
    sw_lng = j - (lng_step /2)
    micro_centers += calculate_grid_centers(sw_lat, sw_lng, ne_lat, ne_lng, grid_divisions=5)

print(micro_centers)
print(len(micro_centers))

[(50.99512297297297, 50.99512297297297), (50.99512297297297, 50.996313742203746), (50.99512297297297, 50.997504511434514), (50.99512297297297, 50.99869528066529), (50.99512297297297, 50.999886049896055), (50.99605972972973, 50.99512297297297), (50.99605972972973, 50.996313742203746), (50.99605972972973, 50.997504511434514), (50.99605972972973, 50.99869528066529), (50.99605972972973, 50.999886049896055), (50.99699648648649, 50.99512297297297), (50.99699648648649, 50.996313742203746), (50.99699648648649, 50.997504511434514), (50.99699648648649, 50.99869528066529), (50.99699648648649, 50.999886049896055), (50.997933243243246, 50.99512297297297), (50.997933243243246, 50.996313742203746), (50.997933243243246, 50.997504511434514), (50.997933243243246, 50.99869528066529), (50.997933243243246, 50.999886049896055), (50.998870000000004, 50.99512297297297), (50.998870000000004, 50.996313742203746), (50.998870000000004, 50.997504511434514), (50.998870000000004, 50.99869528066529), (50.998870000000

In [24]:
all_place_ids = set()
all_places = []
queries = 0
grid = 0
radius = 100
for center in micro_centers:
    print(f"Querying center: {center}")
    params = {
        "location": f"{center[0]},{center[1]}",
        "radius": radius,
        "type": "restaurant",
        "key": API_KEY
    }
    grid += 1
    next_page_token = None
    while True:
        if next_page_token:
            params["pagetoken"] = next_page_token
            time.sleep(2)  # Delay as per Google API guidelines

        data = fetch_results(BASE_URL, params)
        queries += 1
        print("Grid Number: ", grid)
        print("Query Number: ", queries)

        if data['status'] == 'OK':
            all_places.extend(data['results'])

            for place in data["results"]:
                all_place_ids.add(place["place_id"])

            output_file = "resturaunt_ids.json"
            with open(output_file, "w", encoding="utf-8") as f:
                json.dump(list(all_place_ids), f, indent=4)

            output_file = "resturaunts_details.json"
            with open(output_file, "w", encoding="utf-8") as f:
                json.dump(list(all_places), f, indent=4)

            if "next_page_token" in data:
                next_page_token = data.get("next_page_token")
            else:
                break
        else:
            break

Querying center: (50.99512297297297, 50.99512297297297)
Grid Number:  1
Query Number:  1
Querying center: (50.99512297297297, 50.996313742203746)
Grid Number:  2
Query Number:  2
Querying center: (50.99512297297297, 50.997504511434514)
Grid Number:  3
Query Number:  3
Querying center: (50.99512297297297, 50.99869528066529)
Grid Number:  4
Query Number:  4
Querying center: (50.99512297297297, 50.999886049896055)
Grid Number:  5
Query Number:  5
Querying center: (50.99605972972973, 50.99512297297297)
Grid Number:  6
Query Number:  6
Querying center: (50.99605972972973, 50.996313742203746)
Grid Number:  7
Query Number:  7
Querying center: (50.99605972972973, 50.997504511434514)
Grid Number:  8
Query Number:  8
Querying center: (50.99605972972973, 50.99869528066529)
Grid Number:  9
Query Number:  9
Querying center: (50.99605972972973, 50.999886049896055)
Grid Number:  10
Query Number:  10
Querying center: (50.99699648648649, 50.99512297297297)
Grid Number:  11
Query Number:  11
Querying ce